In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_price = StandardScaler().fit_transform(df_market_data)

In [6]:
# Create a DataFrame with the scaled data
# columns = ["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]
columns = ["One_Day", "One_Week", "Two_Weeks", "One_Month", "Two_Months", "200_Days", "One_Year"]

df_market_data_scaled = pd.DataFrame(scaled_price, columns = columns, index = df_market_data.index)

# Copy the crypto names from the original data

# Set the coinid column as index

# Display sample data
df_market_data_scaled.head()

,One_Day,One_Week,Two_Weeks,One_Month,Two_Months,200_Days,One_Year
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [8]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters = i, random_state = 4)
# 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(df_market_data_scaled)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)


In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "Inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

# Display elbow data
elbow_df

,k,Inertia
0,1,287.000000
1,2,198.571818
2,3,123.190482
3,4,79.022435
4,5,68.685015
5,6,55.763391
6,7,48.406314
7,8,37.614301
8,9,32.947547
9,10,29.287118


In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x = "k", y = "Inertia", title = "Elbow Curve", xticks = k)

:Curve   [k]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
k4_model = KMeans(n_clusters = 4, random_state = 4)

In [12]:
# Fit the K-Means model using the scaled data
k4_model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=4)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k4_predictions = k4_model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(k4_predictions)

[3 3 0 0 3 3 3 3 3 0 0 0 0 3 0 3 0 0 3 0 0 3 0 0 0 0 0 0 3 0 0 0 1 3 0 0 2
 0 0 0 0]


In [14]:
# Create a copy of the DataFrame
df_KMeans_predictions = df_market_data_scaled.copy()

In [15]:
df_KMeans_predictions["Feature_1"] = df_KMeans_predictions["One_Day"]

df_KMeans_predictions["Feature_2"] = df_KMeans_predictions["One_Week"]

df_KMeans_predictions["Feature_3"] = df_KMeans_predictions["Two_Weeks"]

In [16]:
df_KMeans_predictions = df_KMeans_predictions.drop(columns = columns)

In [17]:
# Add a new column to the DataFrame with the predicted clusters
df_KMeans_predictions["Cohort"] = k4_predictions

# Display sample data
df_KMeans_predictions.head()

,Feature_1,Feature_2,Feature_3,Cohort
coin_id,,,,
bitcoin,0.508529,0.493193,0.772200,3
ethereum,0.185446,0.934445,0.558692,3
tether,0.021774,-0.706337,-0.021680,0
ripple,-0.040764,-0.810928,0.249458,0
bitcoin-cash,1.193036,2.000959,1.760610,3


In [18]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter1 = df_KMeans_predictions.hvplot.scatter(x = "Feature_1", y = "Feature_2", by = "Cohort", hover_cols = "coin_id", title = "Scatter Plot by Cryptocurrency Cohort where k = 4")
scatter1

:NdOverlay   [Cohort]
   :Scatter   [Feature_1]   (Feature_2,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [19]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components = 3)

In [20]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca_model.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_data_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [21]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_model.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 90% of the total variance in `df_market_data` has been condensed into 3 variables.

In [22]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(market_data_pca, columns = ["PCA1", "PCA2", "PCA3"], index = df_market_data.index)

# Creating a DataFrame with the PCA data


# Copy the crypto names from the original data


# Set the coinid column as index


# Display sample data
df_market_data_pca.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [23]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [24]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters = i, random_state = 4)
# 2. Fit the model to the data using `df_market_data_pca`
    k_model.fit(df_market_data_pca)
# 3. Append the model.inertia_ to the inertia list
    pca_inertia.append(k_model.inertia_)

In [25]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": k, "Inertia_after_PCA": pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)

In [26]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(x = "k", y = "Inertia_after_PCA", title = "Elbow Curve using PCA Data", xticks = k)

:Curve   [k]   (Inertia_after_PCA)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [27]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4, random_state = 4)

In [28]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

KMeans(n_clusters=4, random_state=4)

In [29]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_predictions = model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
pca_predictions

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0])

In [30]:
# Create a copy of the DataFrame with the PCA data
df_PCA_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_PCA_predictions["Cohort"] = pca_predictions

# Display sample data
df_PCA_predictions.head()

,PCA1,PCA2,PCA3,Cohort
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,3


In [31]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter2 = df_PCA_predictions.hvplot.scatter(x = "PCA1", y = "PCA2", by = "Cohort", hover_cols = "coin_id", title = "Scatter Plot by Cryptocurrency Cohort where PCA = 3")
scatter2

:NdOverlay   [Cohort]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [32]:
composite_elbow_df = pd.merge(elbow_df, pca_elbow_df, on = 'k')
composite_elbow_df

,k,Inertia,Inertia_after_PCA
0,1,287.000000,256.874086
1,2,198.571818,168.811896
2,3,123.190482,93.774626
3,4,79.022435,49.665497
4,5,68.685015,37.878747
5,6,55.763391,27.618972
6,7,48.406314,21.145312
7,8,37.614301,17.781191
8,9,32.947547,15.374933
9,10,29.287118,11.607639


In [33]:
# Composite plot to contrast the Elbow curves
composite_elbow_df.hvplot(x = "k", y = ["Inertia", "Inertia_after_PCA"], xticks = k,
                value_label = 'inertia', subplots = True, shared_axes = False, width = 450)

:NdLayout   [Variable]
   :Curve   [k]   (inertia)

In [39]:
scatter1 = df_KMeans_predictions.hvplot.scatter(x = "Feature_1", y = "Feature_2", by = "Cohort", 
            hover_cols = "coin_id", title = "Cryptocurrency Cohort Scatter Plot using Original Data", width = 500)

scatter2 = df_PCA_predictions.hvplot.scatter(x = "PCA1", y = "PCA2", by = "Cohort", 
            hover_cols = "coin_id", title = "Cryptocurrency Cohort Scatter Plot using PCA Data", width = 500)

scatter1 + scatter2


:Layout
   .NdOverlay.I  :NdOverlay   [Cohort]
      :Scatter   [Feature_1]   (Feature_2,coin_id)
   .NdOverlay.II :NdOverlay   [Cohort]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The cluster analysis on the right that used fewer features is less variable and was capable to compiling more comprehensible clusters than the cluster analysis on the left that used more features.